In [235]:
import csv
import glob
import pandas
import numpy as np

# with open(data_file,'r') as csvfile:
#     data = pandas.read_csv(data_file)

In [236]:
with open('../recorder/qualtrics_vids.csv') as csvfile:
    qualtrics_vids = {csvline.split(',')[0]:csvline.split(',')[1].replace('\n','') for csvline in csvfile.readlines()}
#     for csvline in csvfile.readlines():
#         [qualtric,vid] = csvline.split(',')
#         vid = vid[:-1]
#         qualtrics_vids
        
        

In [237]:
df = pandas.read_csv(sorted(glob.glob('./Blossom+Filtering+-+Qualtrics+vids*'))[-1])
# df.loc(list(qualtrics_vids.keys())[0])
orig_names = list(df.columns)
new_names = list(df.iloc[0,:])
# print(new_names.index('Recipient Last Name'))
# new_names[new_names.index('Recipient Last Name')+1:] = 
id_start_idx = orig_names.index('1_Q3.2')
for n,new_name in enumerate(new_names):
    if n<id_start_idx or n==(len(new_names)-1): continue
    new_names[n] = qualtrics_vids[new_names[n].split()[0]]

df = df.rename(columns = {orig_name:new_name for orig_name,new_name in zip(orig_names,new_names)})
df = df.drop([0,1])
drop_columns=['Start Date','End Date','IP Address','Recorded Date','Response ID','Recipient Last Name',
                'Recipient First Name','Recipient Email','External Data Reference','Location Latitude','Location Longitude','Distribution Channel','User Language',
                'Response Type',
             'Progress','Duration (in seconds)','Finished','Please enter your Mechanical Turk ID:','Random ID']
# df = df.drop(columns=[drop_columns])
durations = df['Duration (in seconds)'].astype(int)
for drop_column in drop_columns:
    try: 
        df = df.drop(columns=[drop_column])
    except:
        print(f"Couldnt find {drop_column}")
# df.drop(columns=[df.filter(like='Online Survey Consent')])
df = df[df.columns.drop(list(df.filter(regex='Online Survey Consent')))]
emotions = ['anger','happy','sad']
Emotions = ['Angry','Happy','Sad']
# df = df.replace({Emotion:emotion for Emotion,emotion in zip()})
df = df.replace(dict(zip(Emotions,emotions)))
df = df[:250]
df

,anger_homer_2021228_192746,happy_20181204_123101,sad_pika_202132_163555,happy_sponge_202133_22370,happy_homer_2033_1048,sad_homer_202222_5245,anger_pika_202133_205740,sad_despair,sad_pika_202133_123354,anger_homer_2021228_172045,...,sad_homer_202122_13429,sad_head_down,happy_sponge_20228_152814,happy_pika_202122_13431,happy_pika_013_17434,happy_pika_202133_192916,happy_homer_2132_319,happy_20181204_122642,sad_20181204_122044,sad_pika_2021228_162653
2,NaN,NaN,anger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,anger,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,anger,NaN,NaN,sad,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sad,NaN
5,happy,NaN,NaN,NaN,anger,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,anger,NaN,sad,NaN,NaN,NaN,NaN,NaN,sad,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,anger,anger
248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sad,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,sad,happy,NaN,NaN,NaN
250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,happy,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [240]:
true_accs,any_accs,counts = [],[],[]
vids = list(qualtrics_vids.values())
for vid in vids:
    vid_counts = {emotion:0 for emotion in emotions}
    vid_counts.update(dict(df[vid].value_counts()))
    count_values = list(vid_counts.values())
    if sum(count_values)==0: continue
#     print(sum(count_values))
    counts.append(sum(count_values))
    vid_counts.update({vid:(count/sum(count_values)) for vid,count in vid_counts.items()})
    true_accs.append(vid_counts[vid.split('_')[0]])
    any_accs.append(np.max(list(vid_counts.values())))
#     accs.append(acc)
#     if len(count_values)==0: continue
#     print(list(vid_counts.values()))
#     if len()
#     max_count = np.max(count_values)
    
#     print(vid,vid_counts)
#     print(vid,acc)
print(np.average(counts))
for thresh in [0.33,0.50,0.66,0.80]:
    accs_filt = lambda accs : np.where(np.array(accs)>thresh)[0]
    print(f'{thresh}%  True:{len(accs_filt(true_accs))}  Any:{len(accs_filt(any_accs))}')
    
#     thresh_vids = np.array([])
    thresh_vids = []
    for emotion in emotions:
        emotion_idxs = [v for v,vid in enumerate(vids) if emotion in vid]
        emotion_idxs = [emotion_idx for emotion_idx in emotion_idxs if emotion_idx in accs_filt(true_accs)]
#         thresh_vids = np.concatenate([thresh_vids,np.array(vid)[emotion_idxs]])
        [thresh_vids.append(vid.replace('anger','angry')) for vid in np.array(vids)[emotion_idxs]]
    #     print(emotion_idxs)
        print(f'{emotion} accuracy: {np.average(np.array(true_accs)[emotion_idxs])}, number: {len(emotion_idxs)}')
    with open(f'./thresholds/vids_{str(int(thresh*100))}.txt','w') as thresh_file:
        thresh_file.write('\n'.join(thresh_vids))
#     print(thresh_vids)

24.41176470588235
0.33%  True:147  Any:204
anger accuracy: 0.48331426336237915, number: 36
happy accuracy: 0.5814344135275387, number: 52
sad accuracy: 0.667372731722182, number: 59
0.5%  True:91  Any:136
anger accuracy: 0.6022602771132888, number: 13
happy accuracy: 0.6848642599756629, number: 31
sad accuracy: 0.7283678088949466, number: 47
0.66%  True:49  Any:65
anger accuracy: nan, number: 0
happy accuracy: 0.7860616070617367, number: 14
sad accuracy: 0.7771189977425268, number: 35
0.8%  True:16  Any:19
anger accuracy: nan, number: 0
happy accuracy: 0.9157943349753694, number: 4
sad accuracy: 0.8698812447643723, number: 12


In [239]:
durations.to_frame().mean()/60


Duration (in seconds)    16.007745
dtype: float64

In [231]:
!rm vids_*